**Late submission**
- early stopping for each target

In [1]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd 
from umap import UMAP
import networkx as nx
from sklearn import preprocessing
from sklearn.metrics import log_loss,roc_auc_score
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import KMeans
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import make_column_transformer,ColumnTransformer
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.pipeline import make_pipeline,make_union
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold, KFold
from tqdm._tqdm_notebook import tqdm_notebook

sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from torch.nn.modules.loss import _WeightedLoss

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`


# final engineering

In [2]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')
drug = pd.read_csv(DATA_DIR + 'train_drug.csv')

In [3]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [4]:
noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[train.cp_type!="ctl_vehicle"].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# target group

In [5]:
y = targets.drop("sig_id", axis=1)

# preprocess

In [6]:
train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
test = test[test.index.isin(cons_test_index)].copy().reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)

# Feature engineering 

In [7]:
#importance = pd.read_csv('../input/moa-feat-importance-rapids/output_source.csv')
#importance = importance.set_index("Feature")

#imp_scaled = importance.copy(deep=True)
#for c in imp_scaled.columns:
#    imp_scaled[c] = (1 - preprocessing.MinMaxScaler().fit_transform(imp_scaled[[c]])).round(14)

#print(imp_scaled.min().min(), imp_scaled.max().max(), imp_scaled.std().mean())

#imp_scaled["MeanImp"] = imp_scaled[target_feats].mean(axis=1)
#imp_scaled["MaxImp"]  = imp_scaled[target_feats].max(axis=1)

#thresh_mean, thresh_max = 0.3, 0.95
#fs_both = imp_scaled.loc[(imp_scaled.MeanImp >= thresh_mean) & (imp_scaled.MaxImp >= thresh_max), Targets]
#fs_any = list(imp_scaled.loc[(imp_scaled.MeanImp >= thresh_mean) | (imp_scaled.MaxImp >= thresh_max), target_feats].index)
#print(len(fs_any))

#least_contribution = list(imp_scaled.sort_values("MeanImp", ascending=True).index)
#least_contribution2 = list(imp_scaled.sort_values("MaxImp", ascending=True).index)
#drop_feats = [i for i in c_feats+g_feats if i not in fs_any]

X = train.iloc[:,4:].copy().values
select = VarianceThreshold(threshold=0.7)
X_new = select.fit_transform(X)
drop_feats = list(np.array(train.iloc[:,4:].columns)[select.get_support()==False])
print(len(drop_feats))

#drop_feats = least_contribution[:10]
#drop_feats += least_contribution2[:10]

train.drop(drop_feats, axis=1, inplace=True)
test.drop(drop_feats, axis=1, inplace=True)

g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

33


In [8]:
# rank gauss
for i in c_feats + g_feats:
    ss = preprocessing.QuantileTransformer(n_quantiles=1000, random_state=0, output_distribution="normal")
    #    ss.fit(pd.concat([train[i], test[i]]).values.reshape(-1,1))
    ss.fit(train[i].values.reshape(-1,1))
    train[i] = ss.transform(train[i].values.reshape(-1,1))
    test[i] = ss.transform(test[i].values.reshape(-1,1))

In [9]:
c_num = 10
pca_c_cols = ["pca-c"+str(i+1) for i in range(c_num)]
pca = PCA(n_components=c_num,random_state=42)
c_train = pca.fit_transform(train[c_feats])
c_test = pca.transform(test[c_feats])
c_train = pd.DataFrame(c_train, columns=pca_c_cols)
c_test = pd.DataFrame(c_test, columns=pca_c_cols)

g_num = 60
pca_g_cols = ["pca-g"+str(i+1) for i in range(g_num)]
pca = PCA(n_components=g_num, random_state=42)
g_train = pca.fit_transform(train[g_feats])
g_test = pca.transform(test[g_feats])
g_train = pd.DataFrame(g_train, columns=pca_g_cols)
g_test = pd.DataFrame(g_test, columns=pca_g_cols)

train = pd.concat([train, c_train],axis=1)
test = pd.concat([test, c_test],axis=1)
train = pd.concat([train, g_train],axis=1)
test = pd.concat([test, g_test],axis=1)

In [10]:
def fe(df):
    tmp = df.copy()
    tmp['g_kurt'] = tmp[g_feats].kurtosis(axis = 1)
    tmp['g_skew'] = tmp[g_feats].skew(axis = 1)
    tmp['c_kurt'] = tmp[c_feats].kurtosis(axis = 1)
    tmp['c_skew'] = tmp[c_feats].skew(axis = 1)
    tmp = pd.get_dummies(tmp, columns=['cp_time','cp_dose'])
    tmp.drop(["cp_type", "sig_id"], axis=1, inplace=True)
    return tmp

train = fe(train)
test = fe(test)

print(train.shape, test.shape)

(21948, 918) (3624, 918)


In [11]:
fn_train = train.copy().to_numpy()
fn_test = test.copy().to_numpy()
fn_targets = targets.drop("sig_id", axis=1).copy().to_numpy()

# modelling

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
class SmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets, n_classes, smoothing=0.0):
        assert 0 <= smoothing <= 1
        with torch.no_grad():
            targets = targets * (1 - smoothing) + torch.ones_like(targets).to(device) * smoothing / n_classes
        return targets

    def forward(self, inputs, targets):
        targets = SmoothCrossEntropyLoss()._smooth(targets, inputs.shape[1], self.smoothing)

        if self.weight is not None:
            inputs = inputs * self.weight.unsqueeze(0)

        loss = F.binary_cross_entropy_with_logits(inputs, targets)

        return loss

In [14]:
print(device)
def seed_everything(seed=42): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

class MoaModel(nn.Module):
    def __init__(self, num_columns, last_num):
        super(MoaModel, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_columns)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_columns, 1024))
        self.relu1 = nn.LeakyReLU()
        
        self.batch_norm2 = nn.BatchNorm1d(1024)
        self.dropout2 = nn.Dropout(0.1)
        self.dense2 = nn.utils.weight_norm(nn.Linear(1024, 1024))
        self.relu2 = nn.LeakyReLU()
        
        self.batch_norm3 = nn.BatchNorm1d(1024)
        self.dropout3 = nn.Dropout(0.1)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1024, last_num))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = self.relu1(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = self.relu2(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

cuda


# modelling

In [15]:
batch_size = 128
n_folds=5
EARLY_STOPPING_STEPS = 10
smoothing = 0.001
p_min = smoothing
p_max = 1 - smoothing
train_epochs = 20

def mean_log_loss(y_true, y_pred):
    metrics = []
    for i, target in enumerate(target_feats):
        metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1]))
    return np.mean(metrics)

def calc_each_loss(y_true, y_pred):
    ans = np.zeros(206)
    for i, target in enumerate(target_feats):
        ans[i] = log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1])
    return ans
    
def modelling_torch(tr, target, te, sample_seed, init_num, last_num):
    seed_everything(seed=sample_seed) 
    X_train = tr.copy()
    y_train = target.copy()
    X_test = te.copy()
    test_len = X_test.shape[0]
    
    mskf=MultilabelStratifiedKFold(n_splits = n_folds, shuffle=True, random_state=224)
    metric = lambda inputs, targets : F.binary_cross_entropy((torch.clamp(torch.sigmoid(inputs), p_min, p_max)), targets)

    models = []
    
    X_test2 = torch.tensor(X_test, dtype=torch.float32)
    test = torch.utils.data.TensorDataset(X_test2) 
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    
    best_oof = pd.DataFrame(np.zeros([len(X_train),y_train.shape[1]]))
    pred_value = np.zeros([test_len, y_train.shape[1]])
    scores = []

    for fold, (train_index, valid_index) in enumerate(mskf.split(X_train, y_train)):
        print("Fold "+str(fold+1))
        X_train2 = torch.tensor(X_train[train_index,:], dtype=torch.float32)
        y_train2 = torch.tensor(y_train[train_index], dtype=torch.float32)

        X_valid2 = torch.tensor(X_train[valid_index,:], dtype=torch.float32)
        y_valid2 = torch.tensor(y_train[valid_index], dtype=torch.float32)
        
        train = torch.utils.data.TensorDataset(X_train2, y_train2)
        valid = torch.utils.data.TensorDataset(X_valid2, y_valid2)
        
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True) 
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
            
        clf = MoaModel(init_num, last_num)
        loss_fn = SmoothCrossEntropyLoss(smoothing=smoothing)

        optimizer = optim.Adam(clf.parameters(), lr = 0.001, weight_decay=1e-5) 
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=train_epochs, steps_per_epoch=len(train_loader))
        
        clf.to(device)
        
        best_val_loss = np.inf
        best_each_val_loss = np.array([np.inf for _ in range(206)])
        test_preds = np.zeros([test_len, y_train.shape[1]])
        
        for epoch in range(train_epochs):
            start_time = time.time()
            clf.train()
            avg_loss = 0.
            sm_avg_loss = 0.
            for x_batch, y_batch in tqdm(train_loader, disable=True):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch) 
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()
                avg_loss += loss.item() / len(train_loader)  
                sm_avg_loss += metric(y_pred, y_batch) / len(train_loader)
                
            clf.eval()
            avg_val_loss = 0.
            sm_avg_val_loss = 0.
            oof_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
            target_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
            for i, (x_batch, y_batch) in enumerate(valid_loader): 
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch).detach()
                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
                sm_avg_val_loss += metric(y_pred, y_batch) / len(valid_loader)
                oof_epoch[i * batch_size:(i+1) * batch_size,:] = torch.clamp(torch.sigmoid(y_pred.cpu()), p_min, p_max)
                target_epoch[i * batch_size:(i+1) * batch_size,:] = y_batch.cpu().numpy()
            each_sm_avg_loss = calc_each_loss(target_epoch, oof_epoch)
        
            elapsed_time = time.time() - start_time 
                    
            mask = best_each_val_loss > each_sm_avg_loss
            best_each_val_loss[mask] = each_sm_avg_loss[mask]
            best_oof.iloc[valid_index,mask] = oof_epoch[:, mask]
            #if sm_avg_val_loss < best_val_loss:
            if sum(mask) > 0:
                best_val_loss = sm_avg_val_loss
                print('Epoch {} imp_mask {} l={:.5f}  val_l={:.5f}  sm_l={:.5f}  sm_val_l={:.5f}  t={:.2f}s'.format(
                    epoch + 1, sum(mask), avg_loss, avg_val_loss, sm_avg_loss, sm_avg_val_loss, elapsed_time))
        
            # test predcition --------------
                for i, (x_batch,) in enumerate(test_loader): 
                    x_batch = x_batch.to(device)
                    y_pred = clf(x_batch).detach()
                    test_preds[i * batch_size:(i+1) * batch_size, mask] = torch.clamp(torch.sigmoid(y_pred.cpu()), p_min, p_max)[:,mask]
        pred_value += test_preds / n_folds
            # ------------------------------
    
    print("Seed {}".format(seed_))
    print("log loss: {}".format(np.mean(best_each_val_loss)))
    
    return best_oof, pred_value

In [16]:
%%time
seeds = [0,1,2,3,4]
target_pred = np.zeros([len(fn_test),fn_targets.shape[1]])

for seed_ in seeds:
    oof, pytorch_pred = modelling_torch(fn_train, fn_targets, fn_test, seed_, fn_train.shape[1], fn_targets.shape[1])
    oof = oof.to_numpy()
    target_oof += oof / len(seeds)
    target_pred += pytorch_pred / len(seeds)

Fold 1
Epoch 1 imp_mask 206 l=0.42385  val_l=0.02236  sm_l=0.42380  sm_val_l=0.02236  t=2.02s
Epoch 2 imp_mask 62 l=0.02038  val_l=0.02874  sm_l=0.02032  sm_val_l=0.02846  t=1.23s
Epoch 3 imp_mask 197 l=0.01837  val_l=0.01784  sm_l=0.01837  sm_val_l=0.01796  t=1.47s
Epoch 4 imp_mask 94 l=0.01760  val_l=0.01783  sm_l=0.01769  sm_val_l=0.01795  t=1.22s
Epoch 5 imp_mask 106 l=0.01736  val_l=0.01746  sm_l=0.01747  sm_val_l=0.01758  t=1.22s
Epoch 6 imp_mask 58 l=0.01723  val_l=0.01796  sm_l=0.01735  sm_val_l=0.01806  t=1.26s
Epoch 7 imp_mask 56 l=0.01741  val_l=0.01740  sm_l=0.01752  sm_val_l=0.01751  t=1.17s
Epoch 8 imp_mask 33 l=0.01727  val_l=0.01748  sm_l=0.01738  sm_val_l=0.01760  t=1.15s
Epoch 9 imp_mask 22 l=0.01718  val_l=0.01739  sm_l=0.01730  sm_val_l=0.01751  t=1.17s
Epoch 10 imp_mask 43 l=0.01712  val_l=0.01697  sm_l=0.01723  sm_val_l=0.01710  t=1.17s
Epoch 11 imp_mask 43 l=0.01685  val_l=0.01702  sm_l=0.01698  sm_val_l=0.01714  t=1.31s
Epoch 12 imp_mask 52 l=0.01669  val_l=0.01

NameError: name 'target_oof' is not defined

In [17]:
t = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
train_checkscore = t.copy()
train_checkscore.loc[train_checkscore.index.isin(cons_train_index),target_feats] = target_oof
train_checkscore.loc[train_checkscore.index.isin(noncons_train_index),target_feats] = 0
t.drop("sig_id", axis=1, inplace=True)
print('OOF log loss: ', log_loss(np.ravel(t), np.ravel(np.array(train_checkscore.iloc[:,1:]))))

NameError: name 'target_oof' is not defined

In [18]:
train_checkscore.to_csv("mlp_oof.csv", index=False)

In [19]:
sub.loc[cons_test_index,target_feats] = target_pred
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)